# Day 2 課題

### 目次
### - 1. Day 1 の作業内容
### - 2. Day 1で構築した予測モデルの汎化性能検証
### - 3. 前処理について
### - 4. サポートベクタ―マシンによる予測モデル構築

## 1. Day 1 の作業内容

### 各変数について
- ID：不要と判断し削除
- name：用いる変数の中ではnameにのみ欠損値があった。欠損値が少ないため、削除して対応した。
- name_length：nameの単語数と成功確率に相関があるのではないかと考え、nameの単語数の変数を追加した。
- category：カテゴリーの種類によって成功確率に違いが見られた。ダミー変数を追加し予測に用いた。データ数の少ないカテゴリーもあり、予測に用いた場合、過学習の懸念がある。
- main_category：成功確率との関係が見られた。ダミー変数を追加し、予測に用いた。
- currency：成功確率との関係が見られた。ダミー変数を追加し、予測に用いた。
- country：成功確率との関係が見られた。ダミー変数を追加し、予測に用いたが、データ数の少ないカテゴリーもあり、予測に用いた場合、過学習の懸念がある。
- goal, usd_goal_real：分布について確認すると、goalの最大値100000000.0をとっているデータが多く見られた。これらは正常な値である可能性もあるが、設定された上限の値を超えていて、便宜的に100000000.0となっている可能性がある。そのためgoalがこれらの値となっているデータについては除外することとする。goalよりも通貨が揃っているusd_goal_realの方が適切と判断し、こちらを予測に用いている。
- deadline, launched：開始から、終了までの日数を求め、変数deltaとして追加した。deltaの外れ値について確認し、最大値91日のものがいくつか見られたが、クラウドファンディングの設定可能日数が91日であり、そのまま使用して問題ないと判断した。日数が0のデータも存在したが、実際に開始日と終了日を確認したところ、期間が一日未満であったため、こちらも問題ないと判断した。
- pledged, usd_peldged, usd_pledged_real, backers：クラウドファンディングの結果得られる値のため、今回は使用不可として削除
- state：目的変数、今回は結果の確定しているsuccessful, failedのみを用い、その他の値となっているデータは扱わないこととした。successful=1, failed=0 として数値に置き換えた。 

量的変数に関しては、標準化して計算に用いた。各変数名に_stdを付けたものを新たにdata frameに追加している。

### 各変数の相関
heatmup, 散布図行列を用いて、各変数の相関を調べた。
- name_lengthとstateには正の相関がみられ、名前の情報量が多い方が、成功しやすい傾向が示唆された。
- usd_goal_realとstateの相関係数は小さかった。目標金額と成功確率は関係しているように思われるが、相関係数には現れていない可能性もある。
- usd_goal_realとdeltaには小さな負の相関が見られた。直感に反するが、関係が相関係数で表現できていない可能性もある。

### ロジスティック回帰による予測
以下の二通の説明変数を用いて、stateの値を予測し、訓練誤差を評価した。
1. name_length_std, usd_goal_std, currencyおよび main_categoryのダミー変数, delta_std 
2. 上記にcountry, categoriのダミー変数を追加したもの
1より2の方が予測精度は高かったが、訓練データに過学習している可能性もあるので、汎化性能を評価したい。

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [4]:
#各質的変数のダミー変数を追加する前のdf
df_day2_1=pd.read_csv('Day1.csv').drop(['usd_goal_real_std', 'name_length_std', 'delta_std'], axis=1)
df_day2_1.head()

,name,category,main_category,currency,deadline,goal,launched,state,country,usd_goal_real,name_length,delta
0,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 00:00:00,50000.0,2016-02-26 13:38:27,1,US,50000.00,3,34
1,Support Solar Roasted Coffee & Green Energy! ...,Food,Food,USD,2014-12-21 00:00:00,1000.0,2014-12-01 18:30:44,1,US,1000.00,8,19
2,Lisa Lim New CD!,Indie Rock,Music,USD,2013-04-08 00:00:00,12500.0,2013-03-09 06:42:58,1,US,12500.00,4,29
3,Tombstone: Old West tabletop game and miniatur...,Tabletop Games,Games,GBP,2017-05-03 00:00:00,5000.0,2017-04-05 19:44:18,1,GB,6469.73,9,27
4,Mike Corey's Darkness & Light Album,Music,Music,USD,2012-08-17 00:00:00,250.0,2012-08-02 14:11:32,1,US,250.00,6,14


In [7]:
#currency, main_ategoryのダミー変数を追加したdf
df_day2_2=pd.read_csv('Day1_2.csv').drop(['usd_goal_real_std', 'name_length_std', 'delta_std'], axis=1)
df_day2_2.head()

,name,category,deadline,goal,launched,state,country,usd_goal_real,name_length,delta,...,currency_EUR,currency_GBP,currency_HKD,currency_JPY,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD
0,Monarch Espresso Bar,Restaurants,2016-04-01 00:00:00,50000.0,2016-02-26 13:38:27,1,US,50000.00,3,34,...,0,0,0,0,0,0,0,0,0,1
1,Support Solar Roasted Coffee & Green Energy! ...,Food,2014-12-21 00:00:00,1000.0,2014-12-01 18:30:44,1,US,1000.00,8,19,...,0,0,0,0,0,0,0,0,0,1
2,Lisa Lim New CD!,Indie Rock,2013-04-08 00:00:00,12500.0,2013-03-09 06:42:58,1,US,12500.00,4,29,...,0,0,0,0,0,0,0,0,0,1
3,Tombstone: Old West tabletop game and miniatur...,Tabletop Games,2017-05-03 00:00:00,5000.0,2017-04-05 19:44:18,1,GB,6469.73,9,27,...,0,1,0,0,0,0,0,0,0,0
4,Mike Corey's Darkness & Light Album,Music,2012-08-17 00:00:00,250.0,2012-08-02 14:11:32,1,US,250.00,6,14,...,0,0,0,0,0,0,0,0,0,1


## 2. Day 1で構築した予測モデルの汎化性能検証
#### 交差検証を用いて汎化性能を評価する

In [8]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix 
from sklearn.model_selection import KFold 

### 2-1 name_length_std, usd_goal_std, delta_std, currencyおよび main_categoryのダミー変数を説明変数とした予測モデルの評価

In [29]:
#必要ない変数を除く
X = df_day2_2.drop(["state", "launched", "country", "name", "category", "deadline", "goal"], axis=1)
y = df_day2_2["state"]
X.head()

,usd_goal_real,name_length,delta,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,main_category_Film & Video,...,currency_EUR,currency_GBP,currency_HKD,currency_JPY,currency_MXN,currency_NOK,currency_NZD,currency_SEK,currency_SGD,currency_USD
0,50000.00,3,34,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1000.00,8,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,12500.00,4,29,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,6469.73,9,27,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,250.00,6,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [32]:
# 交差検証法により、汎化性能を検証する。
n_split = 5 #データの分割数
result=[]#各回の正答率を格納する配列
split_num = 1

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X.iloc[train_idx, :], y[train_idx] #学習用データ
    X_test, y_test = X.iloc[test_idx, :], y[test_idx]     #テスト用データ
    stdc =  StandardScaler()
    stdc.fit(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])
    X_train.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])                                                                    
    X_test.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_test.loc[:, ["usd_goal_real", "name_length", "delta"]])
    #学習用データを用いてモデルの学習を行う。
    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)

    # 訓練データのラベルを予測
    y_est = clf.predict(X_train)
    # 確率値を得る
    y_est_proba = clf.predict_proba(X_train)
    # 対数尤度を計算
    llr = - log_loss(y_train, y_est_proba, normalize=False)
    # 正答率を計算
    accuracy=100 * accuracy_score(y_train, y_est)

    # Precision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_est)

    result.append(["Fold %s 訓練"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])

    # テストデータのラベルを予測
    y_est = clf.predict(X_test)
    # 確率値を得る
    y_est_proba = clf.predict_proba(X_test)    
    # 対数尤度を計算
    llr = - log_loss(y_test, y_est_proba, normalize=False)
    # 正答率を計算
    accuracy=100 * accuracy_score(y_test, y_est)
    # Precision, Recall, F1-scoreを計算
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est)
   
    result.append(["Fold %s テスト"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])
    split_num += 1

#各回のデータが入った配列をdataframeにし、訓練データ、テストデータそれぞれで各値の平均を求める。
result_df = pd.DataFrame(result,columns=['name', '対数尤度', '正答率', '適合率', '再現率', 'F1値'],)
result_df.set_index('name')
display(result_df)
test_result = result_df.iloc[[1, 3, 5, 7, 9], : ]
train_result = result_df.iloc[[0, 2, 4, 6, 8], : ]

print("訓練データ Cross Validation")
display(train_result.mean())

print("テストデータ Cross Validation")
display(test_result.mean())


/opt/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


,name,対数尤度,正答率,適合率,再現率,F1値
0,Fold 1 訓練,-164945.404032,64.941164,59.352241,42.428730,49.483517
1,Fold 1 テスト,-41355.652521,64.801218,58.422697,42.218711,49.016225
2,Fold 2 訓練,-165045.796118,65.110397,59.365276,42.941330,49.834982
3,Fold 2 テスト,-41410.239041,65.034902,59.527006,42.883714,49.852967
4,Fold 3 訓練,-165138.121305,64.943426,59.172754,42.169553,49.244756
5,Fold 3 テスト,-41247.506808,64.995703,59.799759,42.319249,49.563366
6,Fold 4 訓練,-165309.861417,64.701958,58.208644,44.308860,50.316454
7,Fold 4 テスト,-41351.999666,64.610722,58.458527,44.359450,50.442310
8,Fold 5 訓練,-165384.302869,64.837646,58.497407,45.084395,50.922463
9,Fold 5 テスト,-41097.224184,65.259016,58.719820,45.068411,50.996321


訓練データ Cross Validation


対数尤度   -165164.697148
正答率         64.906918
適合率         58.919265
再現率         43.386574
F1値         49.960434
dtype: float64

テストデータ Cross Validation


対数尤度   -41292.524444
正答率        64.940312
適合率        58.985562
再現率        43.369907
F1値        49.974238
dtype: float64

訓練データにおける精度、テストデータにおける誤差がほぼ変わらず、過学習は起こしていないように思われる。対数尤度の合計値も訓練データがテストデータの4倍の量であるから、合計値がほぼ4倍になっており、差はほとんど見られない。

### 2-2 country, categoryのダミー変数を説明変数に追加した予測モデルの評価

In [33]:
#ダミー変数の追加
df_day2_3=pd.get_dummies(df_day2_2, columns=["country"])
df_day2_3=pd.get_dummies(df_day2_3, columns=["category"])

In [35]:
#必要ない変数を除く
X2 = df_day2_3.drop(["state", "launched", "name",  "deadline", "goal"], axis=1)
y2 = df_day2_3["state"]
X2.head()

,usd_goal_real,name_length,delta,main_category_Art,main_category_Comics,main_category_Crafts,main_category_Dance,main_category_Design,main_category_Fashion,main_category_Film & Video,...,category_Wearables,category_Weaving,category_Web,category_Webcomics,category_Webseries,category_Woodworking,category_Workshops,category_World Music,category_Young Adult,category_Zines
0,50000.00,3,34,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1000.00,8,19,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,12500.00,4,29,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6469.73,9,27,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,250.00,6,14,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
# 交差検証法により、汎化性能を検証する。
n_split = 5 
result=[]
split_num = 1

for train_idx, test_idx in KFold(n_splits=n_split, shuffle=True, random_state=1234).split(X, y):
    X_train, y_train = X2.iloc[train_idx, :], y2[train_idx] 
    X_test, y_test = X2.iloc[test_idx, :], y2[test_idx]    
    stdc =  StandardScaler()
    stdc.fit(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])
    X_train.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])                                                                    
    X_test.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_test.loc[:, ["usd_goal_real", "name_length", "delta"]])
    clf = SGDClassifier(loss='log', penalty='none', max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)
    
    y_est = clf.predict(X_train)
    y_est_proba = clf.predict_proba(X_train)
    llr = - log_loss(y_train, y_est_proba, normalize=False)
    accuracy=100 * accuracy_score(y_train, y_est)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_train, y_est)
    result.append(["Fold %s 訓練"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])

    y_est = clf.predict(X_test)
    y_est_proba = clf.predict_proba(X_test)    
    llr = - log_loss(y_test, y_est_proba, normalize=False)
    accuracy=100 * accuracy_score(y_test, y_est)
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_test, y_est)
    result.append(["Fold %s テスト"%split_num, llr, accuracy, 100 * precision[1], 100 * recall[1], 100 * f1_score[1]])

    split_num += 1

result_df = pd.DataFrame(result,columns=['name', '対数尤度', '正答率', '適合率', '再現率', 'F1値'],)
result_df.set_index('name')
display(result_df)
test_result = result_df.iloc[[1, 3, 5, 7, 9], : ]
train_result = result_df.iloc[[0, 2, 4, 6, 8], : ]

print("訓練データ Cross Validation")
display(train_result.mean())

print("テストデータ Cross Validation")
display(test_result.mean())

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


,name,対数尤度,正答率,適合率,再現率,F1値
0,Fold 1 訓練,-159011.195592,67.195851,61.948795,49.103592,54.783304
1,Fold 1 テスト,-39885.331123,67.026489,61.003176,49.136666,54.430670
2,Fold 2 訓練,-159336.298067,67.143460,62.230362,47.285055,53.737933
3,Fold 2 テスト,-39975.271946,67.031012,62.403523,46.919872,53.565210
4,Fold 3 訓練,-159315.837977,67.186805,61.021877,51.590202,55.911071
5,Fold 3 テスト,-39810.324151,67.006890,61.180412,51.489409,55.918137
6,Fold 4 訓練,-159533.229801,66.993197,61.963152,47.073608,53.501758
7,Fold 4 テスト,-39909.070878,66.768182,61.983917,46.940215,53.423210
8,Fold 5 訓練,-159429.534301,67.059910,61.434972,49.938520,55.093390
9,Fold 5 テスト,-39736.049309,67.407430,61.539530,49.973688,55.156820


訓練データ Cross Validation


対数尤度   -159325.219148
正答率         67.115845
適合率         61.719832
再現率         48.998195
F1値         54.605491
dtype: float64

テストデータ Cross Validation


対数尤度   -39863.209481
正答率        67.048001
適合率        61.622111
再現率        48.891970
F1値        54.498810
dtype: float64

テストデータにおける精度がわずかに低くなっているが、明らかに過学習が起きているとは言えない。説明変数の追加によって精度が数%上昇している。

## 3. ロジスティック回帰の正則化項の検討

### 2-2 でより高い精度を実現した説明変数を採用する。
#### ここではホールドアウト法で検証する。

In [37]:
from sklearn.model_selection import train_test_split

In [55]:
#データの分割　正則化項の強さのチューニングを行うため、最終テスト用のテストデータを残している。
X_train, X_final_test, y_train, y_final_test = train_test_split(X2, y2, test_size = 0.1, random_state=1234)
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size = 0.1, random_state=1234)

In [57]:
# 量的変数の標準化
stdc =  StandardScaler()
stdc.fit(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])
X_train.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])                                                                    
X_final_test.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_final_test.loc[:, ["usd_goal_real", "name_length", "delta"]])
X_test.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_test.loc[:, ["usd_goal_real", "name_length", "delta"]])

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [58]:
#予測結果を出力する関数
def print_result(y, y_est, y_est_proba):
    precision, recall, f1_score, _ = precision_recall_fscore_support(y, y_est)
    print('対数尤度 = {:.3f}'.format(- log_loss(y, y_est_proba, normalize=False))) 
    print('正答率 = {:.3f}%'.format(100 * accuracy_score(y, y_est)))
    print('適合率（Precision） = {:.3f}%'.format(100 * precision[1]))
    print('再現率（Recall） = {:.3f}%'.format(100 * recall[1]))
    print('F1値（F1-score） = {:.3f}%'.format(100 * f1_score[1]))
    
    return 

In [59]:
#検証用の関数を定義する。データ、正則化項（l1 or l2),正則化の強さalphaを引数としている。
#戻り値は重みのリスト、対数尤度、正答率、訓練済みモデルである。
def reguralization(X_train, X_test, y_train, y_test, penalty, alpha):
    clf = SGDClassifier(loss='log', penalty=penalty, alpha=alpha,  max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)

    # 重みを取得
    w0 = clf.intercept_[0]
    w = clf.coef_

    # 訓練データのラベルを予測
    y_est = clf.predict(X_train)
    y_est_proba = clf.predict_proba(X_train)
    print("訓練データ")
    print_result(y_train, y_est, y_est_proba)
    
    # テストデータのラベルを予測
    y_est = clf.predict(X_test)
    y_est_proba = clf.predict_proba(X_test)
    print("\nテストデータ")
    print_result(y_test, y_est, y_est_proba)

    return w, - log_loss(y_test, y_est_proba, normalize=False), 100 * accuracy_score(y_test, y_est), clf

In [60]:
#パラメータチューニング用の関数。上記の関数を簡略化したもの。
def reguralization2(X_train, X_test, y_train, y_test, penalty, alpha):
    clf = SGDClassifier(loss='log', penalty=penalty, alpha=alpha,  max_iter=10000, fit_intercept=True, random_state=1234, tol=1e-3, )
    clf.fit(X_train, y_train)

    # 重みを取得
    w0 = clf.intercept_[0]
    w = clf.coef_

    # テストデータのラベルを予測
    y_est = clf.predict(X_test)
    y_est_proba = clf.predict_proba(X_test)

    return  w, - log_loss(y_test, y_est_proba, normalize=False), 100 * accuracy_score(y_test, y_est), clf

比較のため、正則化なしのモデルも同じ訓練データで学習し、最終テストデータでの予測精度を評価する。
重みについても確認する。

In [61]:
w_0, ll_0, a_0, clf_0 = reguralization(X_train, X_final_test, y_train, y_final_test, "none", 1)
w_0

訓練データ
対数尤度 = -173942.032
正答率 = 62.021%
適合率（Precision） = 56.651%
再現率（Recall） = 25.784%
F1値（F1-score） = 35.439%

テストデータ
対数尤度 = -21430.605
正答率 = 62.081%
適合率（Precision） = 55.612%
再現率（Recall） = 25.737%
F1値（F1-score） = 35.189%


array([[-1.23617638e-01,  2.47326819e-01, -2.41841807e-01,
         1.03739001e-01,  8.28948240e-02, -3.40833130e-02,
         2.70694863e-03, -1.05076634e-01, -8.19892433e-02,
        -3.49595136e-02, -5.37554077e-01,  2.01988273e-01,
        -3.97012870e-02,  1.32094317e-01, -2.97865078e-02,
        -3.81238937e-02, -1.90995533e-01,  3.09736655e-01,
        -2.37670245e-02, -6.22464537e-02, -5.41559891e-03,
        -4.99287330e-04, -7.20788852e-02, -2.54877094e-02,
        -1.26030982e-04, -6.62059614e-05, -4.78078869e-03,
        -4.45731374e-03, -1.11067605e-02, -1.11122234e-03,
         9.44314044e-05, -4.80611334e-02, -1.62322575e-03,
        -2.37534329e-02, -3.23998947e-03, -6.22530043e-02,
        -5.41559891e-03, -1.45397350e-02, -5.13567158e-04,
        -1.44712540e-02, -1.20683058e-02, -2.55376971e-02,
        -1.26030982e-04, -1.57088829e-02, -8.85551332e-03,
        -6.62059614e-05, -1.35192031e-05, -4.78078869e-03,
         1.32848080e-04, -1.40798654e-03, -4.43041498e-0

#### まずはL1正則化を試す。

In [62]:
#alpha = 1
w1,ll1,a1,clf1 = reguralization(X_train, X_test, y_train, y_test, "l1", 1)
w1

訓練データ
対数尤度 = -181247.847
正答率 = 59.573%
適合率（Precision） = 0.000%
再現率（Recall） = 0.000%
F1値（F1-score） = 0.000%

テストデータ
対数尤度 = -20148.841
正答率 = 59.485%
適合率（Precision） = 0.000%
再現率（Recall） = 0.000%
F1値（F1-score） = 0.000%


/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

すべてのパラメータの重みがゼロになってしまい、すべて失敗と予測してしまっている。
#### 正則化のパラメータalphaの大きさについて検討してみる。

In [63]:
#alpha = 0.01
w2, ll2, a2, clf2 = reguralization(X_train, X_test, y_train, y_test, "l1", 0.01)

訓練データ
対数尤度 = -176464.621
正答率 = 60.501%
適合率（Precision） = 55.796%
再現率（Recall） = 11.044%
F1値（F1-score） = 18.439%

テストデータ
対数尤度 = -19589.205
正答率 = 60.580%
適合率（Precision） = 56.838%
再現率（Recall） = 11.238%
F1値（F1-score） = 18.766%


In [64]:
#alpha = 0.001
w3, ll3, a3, clf3 = reguralization(X_train, X_test, y_train, y_test, "l1", 0.001)

/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


訓練データ
対数尤度 = -164548.323
正答率 = 66.005%
適合率（Precision） = 61.832%
再現率（Recall） = 41.568%
F1値（F1-score） = 49.714%

テストデータ
対数尤度 = -18226.141
正答率 = 66.202%
適合率（Precision） = 62.427%
再現率（Recall） = 41.644%
F1値（F1-score） = 49.960%


alpha = 0.01, 0.001では学習が上手く行えた。
#### それぞれのalpha値の場合において、予測に用いられた説明変数を確認する。

In [65]:
#説明変数名のリスト, モデルの重みを値に持つdata frameを作成する。
val_df = pd.DataFrame(list(X2.columns),columns=['name'])
val_df['w']=w2.reshape(-1,1)
#重みが大きい順に並べる
val_df = val_df.sort_values('w', ascending=False)
display(val_df.head())
#重みが小さい順に並べる
val_df = val_df.sort_values('w')
display(val_df.head())

,name,w
13,main_category_Music,0.218253
1,name_length,0.180487
158,category_Photobooks,0.000000
146,category_Music Videos,0.000000
136,category_Letterpress,0.000000


,name,w
2,delta,-0.206448
16,main_category_Technology,-0.148936
0,usd_goal_real,-0.014452
105,category_Farmer's Markets,0.000000
14,main_category_Photography,0.000000


In [66]:
val_df2 = pd.DataFrame(list(X2.columns),columns=['name'])
val_df2['w']=w3.reshape(-1,1)
val_df2 = val_df2.sort_values('w', ascending=False)
display(val_df2.head(20))

val_df2 = val_df.sort_values('w')
display(val_df2.head())

,name,w
191,category_Tabletop Games,1.129453
17,main_category_Theater,0.958204
4,main_category_Comics,0.814973
6,main_category_Dance,0.775802
184,category_Shorts,0.666056
13,main_category_Music,0.642139
127,category_Indie Rock,0.322439
3,main_category_Art,0.295784
7,main_category_Design,0.243724
1,name_length,0.237509


,name,w
2,delta,-0.206448
16,main_category_Technology,-0.148936
0,usd_goal_real,-0.014452
136,category_Letterpress,0.000000
137,category_Literary Journals,0.000000


L1正則化の特性通り、予測に用いる説明変数の数が少なくなっていた。少ない説明変数で予測を行う際に参考としたい。
#### 次にパラメータalphaをチューニングする。X_train, y_train, X_test, y_testを用いて最も汎化誤差の少ないパラメータを選び、そのパラメータで最終テストデータを評価し、正則化項なし、L2正則化と比較する。
練習・および計算負荷低下のため、ホールドアウト法で検討しています。交差検証によるパラメータチューニングはSVMの部分で行っています。


In [67]:
#alphaを小さくしながら、モデルの構築を繰り返し、それまでの結果より精度が良かった場合、値とモデルを記憶する。
a = 1
ma = 0
alpha_max = 0
w_max = []
ll_max = -float("inf")
clf_best = 0

for i in range(8):
    w_tmp, ll_tmp, a_tmp, clf_tmp = reguralization2(X_train, X_test, y_train, 
                                   y_test, "l1", a)
    if ll_tmp > ll_max:
        ma = a_tmp
        alpha_max = a
        w_max = w_tmp
        ll_max = ll_tmp
        clf_best = clf_tmp
    a /= 10
    
print('テストデータの誤差が最小となるαは：', alpha_max)    
print('その時の正答率 = {:.3f}%'.format(ma))
print('その時の対数尤度 = {:.3f}'.format(ll_max))

/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


テストデータの誤差が最小となるαは： 0.0001
その時の正答率 = 66.959%
その時の対数尤度 = -17902.131


#### alpha=0.0001が最も汎化誤差が小さくなるパラメータであった。これを用いて最終テストを評価する。

In [68]:
# 最終テストデータのラベルを予測
y_est = clf_best.predict(X_final_test)
y_est_proba = clf_best.predict_proba(X_final_test)
print_result(y_final_test, y_est, y_est_proba)

対数尤度 = -19913.199
正答率 = 67.068%
適合率（Precision） = 61.890%
再現率（Recall） = 45.971%
F1値（F1-score） = 52.755%


正則化項なしの下記のデータと比較すると、精度が向上していることがわかる。
- 対数尤度 = -21428.033
- 正答率 = 62.090%
- 適合率（Precision） = 55.630%
- 再現率（Recall） = 25.775%
- F1値（F1-score） = 35.227%

#### 次にL2正則化を試す

In [69]:
w3, a3, ll4, clf4 = reguralization(X_train, X_test, y_train, y_test, "l2", 1)

訓練データ
対数尤度 = -179629.956
正答率 = 59.574%
適合率（Precision） = 100.000%
再現率（Recall） = 0.001%
F1値（F1-score） = 0.002%

テストデータ
対数尤度 = -19962.212
正答率 = 59.485%
適合率（Precision） = 0.000%
再現率（Recall） = 0.000%
F1値（F1-score） = 0.000%


/opt/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


L2正則化でもalphaが1だとほとんどのデータを失敗と予測していると推測できる。
#### テストデータの正答率を最大にするalphaの値を求めてみる。

In [70]:
a = 1
ma = 0
alpha_max = 0
w_max = []
ll_max = -float("inf")
clf_best = 0

for i in range(8):
    w_tmp, ll_tmp, a_tmp, clf_tmp = reguralization2(X_train, X_test, y_train, 
                                   y_test, "l2", a)
    if ll_tmp > ll_max:
        ma = a_tmp
        alpha_max = a
        w_max = w_tmp
        ll_max = ll_tmp
        clf_best = clf_tmp
    a /= 10
    
print('テストデータの誤差が最小となるαは：', alpha_max)    
print('その時の正答率 = {:.3f}%'.format(ma))
print('その時の対数尤度 = {:.3f}'.format(ll_max))

/opt/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


テストデータの誤差が最小となるαは： 0.0001
その時の正答率 = 66.845%
その時の対数尤度 = -17909.376


alphaがおよそ0.0001程度の大きさでテストデータの正答率が最大となることがわかった。最終テストを評価する。

In [71]:
# テストデータのラベルを予測
y_est = clf_best.predict(X_final_test)
y_est_proba = clf_best.predict_proba(X_final_test)
print_result(y_final_test, y_est, y_est_proba)
w_max

対数尤度 = -19909.622
正答率 = 67.019%
適合率（Precision） = 61.773%
再現率（Recall） = 46.023%
F1値（F1-score） = 52.748%


array([[-6.14340676e+00,  2.11473020e-01, -2.96362693e-01,
         1.25625793e-01,  7.37692283e-01, -4.30325840e-01,
         5.33870797e-01,  4.75501831e-03, -4.80617475e-01,
         4.61818415e-02, -3.97120197e-01, -2.05067712e-01,
        -2.35012936e-01,  3.71406132e-01, -4.02296000e-01,
        -9.68016182e-02, -2.05565909e-01,  7.02447079e-01,
        -1.32905114e-01,  1.66691101e-02,  1.94722327e-02,
         1.05046028e-01, -1.68896722e-01,  1.28255919e-01,
         1.58584332e-01, -1.37585373e-02, -1.68149060e-01,
        -6.56805715e-02, -3.62071265e-02,  3.22418282e-02,
         3.16190012e-02,  1.62879937e-01, -1.29246812e-01,
        -1.07498744e-01,  1.03608304e-01, -1.15545142e-02,
         1.94722327e-02,  4.69123205e-02,  7.43673626e-02,
        -1.60130745e-01,  4.02019737e-01, -2.89145062e-02,
         1.58584332e-01,  1.20765561e-02, -3.68480358e-01,
        -1.37585373e-02,  1.16681563e-01, -1.68149060e-01,
         1.87258255e-01, -1.00391382e-01, -5.74226741e-0

L1正則化項を加えたものとほぼ同程度の精度が得られた。正則化項なしのものよりこちらでも精度が向上している。正則化項なしの場合でも係数の大きなものはなかったためか、係数の値が小さくなる、というL2正則化の特徴はあまり見られていない。

## 3.  前処理について

#### Day 1にて、追加したダミー変数以外の予測に用いた変数については標準化を行っており、Day 2の課題では省略する。Day 1にてdelta, usd_goal_real, name_lengthにも相関が見られていないため、無相関化も不要であると判断した。

## 4. サポートベクターマシンによる予測モデル構築

In [76]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.model_selection import train_test_split

#### まずはC=10, kernel="rbf", gamma="scale"でモデルの構築を試みる。
#### 説明変数はロジスティック回帰モデルのL１正則化において重要であった5つを用いたもの、2-1のモデルで用いたものの二つを試す。
#### 計算量を減らすため、訓練データを基のデータの10%, テストデータを9%としている。

In [72]:
#L1正則化で重要であったパラメータを用いる。
X3 = df_day2_3[['main_category_Music', 'name_length', 'delta', 'main_category_Technology', 'usd_goal_real']]

In [73]:
#データの分割
#もとのデータの10%を訓練データとし、残りのデータの内の10%をテストデータとする。
X_train, X_res, y_train, y_res = train_test_split(X3, y, test_size = 0.9, random_state=1234)
X_test, X_res, y_test, y_res = train_test_split(X_res, y_res, test_size = 0.9, random_state=1234)

In [74]:
#量的変数の標準化
stdc =  StandardScaler()
stdc.fit(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])
X_train.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])                                                                    
X_test.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_test.loc[:, ["usd_goal_real", "name_length", "delta"]])

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [78]:
#SVMモデルの学習を行う。
C = 10
clf = SVC(C=C,kernel="rbf",gamma=1)
clf.fit(X_train, y_train) 

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [79]:
# 訓練データのラベルを予測し、精度を表示
y_est = clf.predict(X_train)
target_names = ['failed', 'successful']
print(metrics.classification_report(y_train, y_est,target_names=target_names))
conf_mat_train = pd.DataFrame(confusion_matrix(y_train, y_est), 
                        index=['正解 = failed', '正解 = successful'], 
                        columns=['予測 = failed', '予測 = successful'])

display(conf_mat_train)



             precision    recall  f1-score   support

     failed       0.65      0.86      0.74     19809
 successful       0.59      0.30      0.40     13355

avg / total       0.62      0.63      0.60     33164



,予測 = failed,予測 = successful
正解 = failed,17004,2805
正解 = successful,9328,4027


In [80]:
# テストデータのラベルを予測
y_est = clf.predict(X_test)
target_names = ['failed', 'successful']
print(metrics.classification_report(y_test, y_est,target_names=target_names))
conf_mat_test = pd.DataFrame(confusion_matrix(y_test, y_est), 
                        index=['正解 = failed', '正解 = successful'], 
                        columns=['予測 = failed', '予測 = successful'])

display(conf_mat_test)


             precision    recall  f1-score   support

     failed       0.64      0.85      0.73     17816
 successful       0.57      0.29      0.38     12031

avg / total       0.61      0.62      0.59     29847



,予測 = failed,予測 = successful
正解 = failed,15160,2656
正解 = successful,8550,3481


In [81]:
#データの分割、次は2-1のロジスティック回帰に用いた説明変数をすべて利用する。
X_train, X_res, y_train, y_res = train_test_split(X, y, test_size = 0.9, random_state=1234)
X_test, X_res, y_test, y_res = train_test_split(X_res, y_res, test_size = 0.9, random_state=1234)

In [82]:
#量的変数の標準化
stdc =  StandardScaler()
stdc.fit(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])
X_train.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_train.loc[:, ["usd_goal_real", "name_length", "delta"]])                                                                    
X_test.loc[:, ["usd_goal_real", "name_length", "delta"]] = stdc.transform(X_test.loc[:, ["usd_goal_real", "name_length", "delta"]])

/opt/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [83]:
#モデルの学習
C = 10
clf2 = SVC(C=C,kernel="rbf",gamma=1)
clf2.fit(X_train, y_train) 

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [84]:
# 訓練データのラベルを予測
y_est = clf2.predict(X_train)
target_names = ['failed', 'successful']
print(metrics.classification_report(y_train, y_est,target_names=target_names))
conf_mat_train = pd.DataFrame(confusion_matrix(y_train, y_est), 
                        index=['正解 = failed', '正解 = successful'], 
                        columns=['予測 = failed', '予測 = successful'])

display(conf_mat_train)



             precision    recall  f1-score   support

     failed       0.70      0.83      0.76     19809
 successful       0.65      0.47      0.54     13355

avg / total       0.68      0.68      0.67     33164



,予測 = failed,予測 = successful
正解 = failed,16394,3415
正解 = successful,7090,6265


In [85]:
# テストデータのラベルを予測
y_est = clf2.predict(X_test)
target_names = ['failed', 'successful']
print(metrics.classification_report(y_test, y_est,target_names=target_names))
conf_mat_test = pd.DataFrame(confusion_matrix(y_test, y_est), 
                        index=['正解 = failed', '正解 = successful'], 
                        columns=['予測 = failed', '予測 = successful'])

display(conf_mat_test)


             precision    recall  f1-score   support

     failed       0.66      0.79      0.72     17816
 successful       0.56      0.40      0.47     12031

avg / total       0.62      0.63      0.62     29847



,予測 = failed,予測 = successful
正解 = failed,14059,3757
正解 = successful,7190,4841


#### SVMのモデルを構築することができた。グリッドサーチにより、パラメータを最適化した後、最適なパラメータのモデルで再度学習を行い、ロジスティック回帰と比較する。
#### 説明変数はより汎化誤差の少なかった、2-1のものを利用する。

In [30]:
#グリッドサーチによるパラメータの最適化
parameters = {'kernel':['rbf'], 'C':[0.1, 1, 10], 'gamma':[0.1, 1, 10]} 
model = SVC() 
clf = GridSearchCV(model, parameters, cv=3,)
clf.fit(X_train, y_train)
print(clf.best_params_, clf.best_score_)

{'C': 10, 'gamma': 0.1, 'kernel': 'rbf'} 0.6456700036183813


In [33]:
clf3 = SVC(**clf.best_params_)
clf3.fit(X_train, y_train)
print("識別精度=",clf3.score(X_test, y_test))

識別精度= 0.6394277481823969


In [37]:
import pickle 
with open('SVM model', 'wb') as web:
    pickle.dump(clf3, web)

In [86]:
import pickle
with open('SVM model', 'rb') as web:
    clf4 = pickle.load(web)

パラメータのチューニングを行うことができた。データ数を減らしたためか、汎化誤差が訓練誤差よりも大きくなっており、過学習している可能性があある。パラメータのチューニングも探索範囲がまだ十分ではない可能性もあり、今後さらに精度を上げられる可能性がある。

- ロジスティック回帰　正則化項なし　62%
- ロジスティック回帰　L１正則化     67%
- ロジスティック回帰　L２正則化     67%
- SVM                               64% 

今回行った学習の範囲ではL1, L2正則化項を加えたロジスティック回帰が最も汎化誤差が小さいという結果であった。